# **Digital Signal Processing - Midterm Project**
## **Submitted by:**
| **Figure4 Members** |
|   :----:    |
| *Banta, Ian Noel M.*    |
| *Pabelico, John Paul B.*  |
| *Santander, Mark Adrian P.*   |
| *Villasor, Vince Kazer M.*  |

# Libraries

# Data

## Extraction

In [ ]:
## blablablal

## Transformation

### Data Imputation

In [ ]:
### balblabla

## Load

In [ ]:
# albalbalbl

# Functions

# Analysis Goals:

## 1. What is the periodicity of the data?

FFT haha

In [ ]:
# ahaha

## 2. What are the central tendencies of the data according to time of day, day of week, and in general?

### Time of day

5AM To 6AM

In [ ]:
# haha

1PM To 2PM

In [ ]:
# haha

10PM To 11PM

In [ ]:
# haha

### Day of week

Mondays To Thursdays

In [ ]:
# haha

Fridays To Sundays

In [ ]:
# haha

### In general

Ewan ko

In [ ]:
# haha

## 3. What patterns arise from the data you are analyzing?

### idk haha

In [ ]:
# hahaha

# Conclusion?

## Haha

### hahaha